<a href="https://colab.research.google.com/github/Youchenjiang/AI-Final-Project/blob/change/%E4%BA%BA%E5%B7%A5%E6%99%BA%E6%85%A7%E6%9C%9F%E6%9C%AB_14_%E5%89%8D%E9%9D%A2%E6%96%87%E5%AD%97%E4%BF%AE%E6%94%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 人工智慧期末

## 為什麼結果越好反而變異數越小？

在您的程式碼中，calculate_schedule_score 函數用於計算排班結果的得分，其中包含了以下幾個部分：

ShiftNumberBasicScore： 確保每個班別都有足夠的人員。
Penalty： 懲罰連續輪休和同一天上課時段和午休時段的情況。
continuous_work_penalty： 懲罰連續兩天午休輪班的情況。
variance(day_shift) + variance(later_shift) + variance(grave_shift) + variance(day_off)： 計算四種班別（上課時段、午休時段、放學時段、輪休）的變異數總和。
目標是使 calculate_schedule_score 的返回值越小越好，表示排班結果越好。

為什麼結果越好反而變異數越小？

這是因為在您的程式碼中，變異數被用作衡量排班結果公平性的一個指標。變異數越小，表示每個員工在不同班別的工作天數越平均，排班結果越公平。

當演算法找到一個較好的排班結果時，它會傾向於使每個員工在不同班別的工作天數更平均，從而降低變異數。因此，結果越好，變異數越小。

舉例說明：

假設有兩個員工，需要安排他們在 20 天內輪流值班上課時段、午休時段、放學時段和輪休。

排班結果 1： 員工 1 連續值班 10 天上課時段，然後輪休 10 天；員工 2 輪休 10 天，然後連續值班 10 天上課時段。
排班結果 2： 員工 1 和員工 2 每隔一天輪流值班上課時段、午休時段、放學時段和輪休。
很明顯，排班結果 2 比排班結果 1 更公平。 在排班結果 2 中，每個員工在不同班別的工作天數更平均，變異數更小。

因此，在您的基因演算法中，結果越好，變異數越小，這是因為演算法會傾向於找到更公平的排班結果，而公平性的一個重要指標就是變異數。

In [ ]:
from statistics import *
from numpy import *
from random import *
from operator import itemgetter
from copy import deepcopy

CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.1
ITERATION_TIME = 500    #迭代次數
NUMBER_OF_GENETIC = 100  #基因數量
NUMBER_OF_WORKER = 16    #工作人數
WORK_DAY = 20

# 染色體集合
chromosomes = []
best_genetic = []
best_schedule_score = float('inf') #初始化為正無窮大

In [ ]:
def calculate_shift_score(genetic, needToReachNumber, workType, showValue=False):
  score = 15
  shift_numbers = [0]*20
  # 記錄每天上課時段有多少人排班
  for i in range(len(genetic)):
    for j in range(WORK_DAY):
      if(genetic[i][j] == workType):
          shift_numbers[j] += 1
  day_did_not_reach_number = sum(1 for i in shift_numbers if i < needToReachNumber)
  shift_names = ["上課時段", "午休時段", "放學時段", "輪休"]
  if showValue:
    print(shift_names[workType],'沒有達到',needToReachNumber,'人的天數', day_did_not_reach_number)
  score = score * day_did_not_reach_number
  return score

In [ ]:
# 判斷各種班別是否有達到上班人數標準，然後回傳一個分數
def calculate_total_shift_score(teamSchedule,showValue):
  # 0 上課時段 4人; 1 午休時段 2人; 2 放學時段 1人; 3 輪休
  inclassShiftScore = calculate_shift_score(teamSchedule, 4, 0, showValue)
  lunchbreakShiftScore = calculate_shift_score(teamSchedule, 2, 1, showValue)
  afterschoolShiftScore = calculate_shift_score(teamSchedule, 1, 2, showValue)
  return inclassShiftScore + lunchbreakShiftScore + afterschoolShiftScore

In [ ]:
def penalty_continuous_day_off(genetic_list, number_of_worker, work_day, showValue):
  # 避免安排連續3天或以上的輪休 -> (3 + 3 + 3 + ...)
  penalty = 0
  count = 0
  for i in range(number_of_worker):
    continuous_day_off = 0  # 初始化連續輪休日數
    for j in range(work_day):
      if genetic_list[i][j] == 3:  # 如果是輪休
        continuous_day_off += 1
      else:
        if continuous_day_off >= 3:  # 如果連續輪休日數達到3天或以上
          count += continuous_day_off - 2  # 計算懲罰次數
        continuous_day_off = 0  # 重置連續輪休日數
    if continuous_day_off >= 3:  # 處理最後幾天連續輪休的情況
      count += continuous_day_off - 2
  if(showValue):
    print('連續3天或以上輪休的次數：', count)
  return count * 10  # 根據懲罰次數計算懲罰分數

In [ ]:
def penalty_same_day_class_lunch(genetic_list, number_of_worker, work_day, showValue):
  count = 0
  for i in range(number_of_worker):  # 迭代所有員工
    for j in range(work_day):  # 迭代每天
      if genetic_list[i][j] == 0 and genetic_list[i][j] == 1:  # 檢查當天是否有上課時段和午休時段
        count += 1
        break  # 如果同一天有上課時段和午休時段，跳出內層迴圈
  if showValue:
    print('同一天同時有上課時段和午休時段的次數：', count)
  penalty = count * 2  # 計算懲罰分數
  return penalty

In [ ]:
def penalty_continuous_work(teamSchedule, showValue):
  penalty = 0
  for worker_schedule in teamSchedule:
    for i in range(len(worker_schedule) - 1):
      if worker_schedule[i] == 1 and worker_schedule[i + 1] == 1:  # 檢查是否連續兩天午休輪班
        penalty += 10  # 對連續兩天午休輪班進行懲罰
  if showValue:
    print('連續兩天午休輪班的次數：', penalty // 10)
  return penalty

In [ ]:
def calculate_schedule_score(teamSchedule, showValue=False):
  day_shift = [] #上課時段
  later_shift = []
  grave_shift = []
  day_off = []
  for i in range(NUMBER_OF_WORKER):
    day_shift.append(len([i for i in teamSchedule[i] if i == 0])) # 上課時段
    later_shift.append(len([i for i in teamSchedule[i] if i == 1])) # 午休時段
    grave_shift.append(len([i for i in teamSchedule[i] if i == 2])) # 放學時段
    day_off.append(len([i for i in teamSchedule[i] if i == 3])) # 輪休
  ShiftNumberBasicScore = calculate_total_shift_score(teamSchedule,showValue)
  Penalty = penalty_continuous_day_off(teamSchedule, NUMBER_OF_WORKER, WORK_DAY, showValue)+penalty_same_day_class_lunch(teamSchedule, NUMBER_OF_WORKER, WORK_DAY, showValue)
  continuous_work_penalty = penalty_continuous_work(teamSchedule, showValue)
  target_value = round(variance(day_shift)+variance(later_shift)+variance(grave_shift)+variance(day_off), 10) + ShiftNumberBasicScore + Penalty + continuous_work_penalty
  if showValue:
    print('總變異數: ', target_value )
    shift_names = ["上課時段", "午休時段", "放學時段", "輪休"]
    for day in range(WORK_DAY):
      print(f"日期 {day + 1}:")
      for shift in range(4):
        workers = [i + 1 for i, worker_schedule in enumerate(teamSchedule) if worker_schedule[day] == shift]
        print(f"  {shift_names[shift]}: {workers}")
  return target_value

In [ ]:
def perform_crossover():  # 單點交配
  crossover_if = random()
  if crossover_if > CROSSOVER_RATE:  # 判斷是否要交配
    return
  else:
    # 隨機取兩個個體
    first = randint(0, NUMBER_OF_GENETIC - 1)
    second = randint(0, NUMBER_OF_GENETIC - 1)
    while first == second:
      second = randint(0, NUMBER_OF_GENETIC - 1)
    crossover_genetic_1 = chromosomes[first][:]
    crossover_genetic_2 = chromosomes[second][:]
    # 取得突變位置
    crossover_worker = int(random() * (NUMBER_OF_WORKER - 1))  # 取第幾個工人
    crossover_date = int(random() * (WORK_DAY - 1))  # 取工人第幾天的工作，使用 WORK_DAY
    for i in range(crossover_date):
      temp = crossover_genetic_1[crossover_worker][i]
      crossover_genetic_1[crossover_worker][i] = crossover_genetic_2[crossover_worker][i]
      crossover_genetic_2[crossover_worker][i] = temp
    chromosomes[first] = deepcopy(crossover_genetic_1)
    chromosomes[second] = deepcopy(crossover_genetic_2)
  return

In [ ]:
def perform_mutation(): # 每個染色體隨意找地方把一個值改成另一個
  global chromosomes
  muation_if = random()
  if( muation_if > MUTATION_RATE): # 判斷是否要突變
    return
  else:
    genetic_mutation = int(random() * (NUMBER_OF_GENETIC-1))
    worker_mutation = int(random() * (NUMBER_OF_WORKER-1))
    new_genetic = generate_worker_schedule()[:]
    chromosomes[genetic_mutation][worker_mutation] = new_genetic[:]
    return

In [ ]:
def generate_worker_schedule():
  each_worker = []
  work_type = 0
  workDay = 0 # 工作天數
  holiday = 0 # 輪休數量
  twoWeeksWorkDay = 0 # 2個禮拜的工作天(第1、2周；第2、3周；第3、4周 三種可能)
  lastWeekWorkDay = 0
  for i in range(4):  # 4 週
    for j in range(5):  # 每週 5 天
      if holiday >= 4:
        work_type = int(randint(0, 2))  # 避免連續太多天輪休
      elif workDay >= 6:
        work_type = int(randint(0, 2))  # 避免連續太多天工作
        random_index = randrange(len(each_worker) - 6, len(each_worker))
        each_worker[random_index] = 3  # 調整為輪休
        holiday += 1
        workDay -= 1
      else:  # random時給定一個比例
        type = [0, 0, 0, 0, 1, 1, 1, 2, 2, 3]
        work_type = choices(type)[0]
      if work_type == 3:
        holiday += 1
      elif work_type != 3:
        workDay += 1
      each_worker.append(work_type)
      if i * 5 + j + 1 == 10 or i * 5 + j + 1 == 15 or i * 5 + j + 1 == 20:  # 第二週、第三週、第四週
        twoWeeksWorkDay = lastWeekWorkDay + workDay
        if twoWeeksWorkDay > 11:
          for k in range(twoWeeksWorkDay - 11):
            random_index = randrange(len(each_worker) - 6, len(each_worker))
            while each_worker[random_index] == 3:
              random_index = randrange(len(each_worker) - 6, len(each_worker))
            each_worker[random_index] = 3  # 調整為輪休
            workDay -= 1
            holiday += 1

    lastWeekWorkDay = workDay
    workDay = 0
    holiday = 0

  return each_worker

In [ ]:
def initialize_population():
  for i in range(NUMBER_OF_GENETIC):
    one_genetic = []
    for j in range(NUMBER_OF_WORKER):
      one_genetic.append(generate_worker_schedule()[:])
    chromosomes.append(one_genetic[:])
    # 記錄最好的基因
    if(i == 0):
      best_genetic = one_genetic[:]
      best_schedule_score = calculate_schedule_score(best_genetic)
    elif(best_schedule_score > calculate_schedule_score(one_genetic) ):
      best_genetic = one_genetic[:]
      best_schedule_score = calculate_schedule_score(best_genetic)

In [ ]:
def select_next_generation(number_of_generation):
  global chromosomes
  global best_schedule_score
  global best_genetic
  temp_all_genetic = []
  biggestTargetValue = -1
  population = []

  for j in range(NUMBER_OF_GENETIC):
    targetValue = calculate_schedule_score(chromosomes[j])
    population.append(targetValue)
    if(biggestTargetValue < targetValue):
      biggestTargetValue = targetValue
  # 如果全部基因都一樣沒有交配或突變就不用演進了
  different = True
  for i in range(1,len(population)):
    d = population[0]
    if(d!=population[i]):
      different = False
  if different:
    return
  for i in range(len(population)):
    population[i] = biggestTargetValue - population[i]
  for i in range(NUMBER_OF_GENETIC):
    x = choices( range(len(population)), population )
    temp_all_genetic.append(chromosomes[x[0]][:] )
  chromosomes = temp_all_genetic[:]
  for i in range(NUMBER_OF_GENETIC):
    targetValue = calculate_schedule_score(chromosomes[i])
    # 記錄最好的基因
    if(best_schedule_score > targetValue ):
      best_genetic = deepcopy(chromosomes[i])
      best_schedule_score = calculate_schedule_score(best_genetic)
  if(number_of_generation%10 == 0 or number_of_generation == 1):
    print('\n','======================= ITERATION ', number_of_generation ,' =======================')
    print('\n','----- Best Genetic Target Value: ', best_schedule_score, '-----')
    calculate_schedule_score(best_genetic, True)
  return

In [ ]:
def run_genetic_algorithm():
  initialize_population() # 初始化: 建立所有基因
  for i in range(ITERATION_TIME):
    select_next_generation(i+1) # 選擇: 選擇下一代基因
    mutation_times = 3  # 突變次數
    crossover_times = 3  # 交配次數
    for j in range(mutation_times):
      perform_mutation() # 突變 => 突變一點太小了，這邊直接突變一個工人的全部
    for j in range(crossover_times):
      perform_crossover() # 交配
if __name__ == "__main__":
    run_genetic_algorithm()

<ipython-input-46-1d800df0d5bb>:9: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  day_did_not_reach_number = sum(1 for i in shift_numbers if i < needToReachNumber)



 ======================= ITERATION  1  =======================

 ----- Best Genetic Target Value:  200.4416666667 -----
上課時段 沒有達到 4 人的天數 0
午休時段 沒有達到 2 人的天數 0
放學時段 沒有達到 1 人的天數 0
連續3天或以上輪休的次數： 0
同一天同時有上課時段和午休時段的次數： 0
連續兩天午休輪班的次數： 19
總變異數:  200.4416666667
日期 1:
  上課時段: [3, 6, 8, 9, 11, 12, 13, 14]
  午休時段: [2, 4, 7, 15]
  放學時段: [1, 5, 10, 16]
  輪休: []
日期 2:
  上課時段: [4, 5, 6, 9, 12, 14]
  午休時段: [10, 13, 16]
  放學時段: [2, 3, 7, 11, 15]
  輪休: [1, 8]
日期 3:
  上課時段: [3, 5, 7, 8, 11, 15, 16]
  午休時段: [2, 10, 13, 14]
  放學時段: [1, 4, 6, 12]
  輪休: [9]
日期 4:
  上課時段: [3, 5, 10, 12, 13, 14]
  午休時段: [2, 8, 9, 11]
  放學時段: [1, 4, 15]
  輪休: [6, 7, 16]
日期 5:
  上課時段: [3, 5, 6, 9, 11, 13, 14, 16]
  午休時段: [2, 4, 7]
  放學時段: [1, 10]
  輪休: [8, 12, 15]
日期 6:
  上課時段: [3, 4, 5, 6, 8, 9, 10, 12]
  午休時段: [1, 2, 11, 14]
  放學時段: [7, 13, 15, 16]
  輪休: []
日期 7:
  上課時段: [2, 3, 8, 10, 11, 14, 15]
  午休時段: [4, 5, 6, 9, 12]
  放學時段: [1, 7, 13, 16]
  輪休: []
日期 8:
  上課時段: [2, 5, 7, 10, 12]
  午休時段: [1, 3, 6, 8, 9, 15, 16]
  放學時段: [11